In [2]:
!pip install telnetlib

ERROR: Could not find a version that satisfies the requirement telnetlib (from versions: none)
ERROR: No matching distribution found for telnetlib

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [3]:
!pip install modexp

ERROR: Could not find a version that satisfies the requirement modexp (from versions: none)
ERROR: No matching distribution found for modexp

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [5]:
!pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 3.7 MB/s eta 0:00:00a 0:00:010m
  Attempting uninstall: pip
    Found existing installation: pip 23.3.1
    Uninstalling pip-23.3.1:
      Successfully uninstalled pip-23.3.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
python-dev-tools 2023.3.24 requires pip<23,>=22, but you have pip 23.3.2 which is incompatible.


In [7]:
import telnetlib
import json

# Implementación de la función modexp
def modexp(base, exponent, modulus):
    result = 1
    base = base % modulus
    while exponent > 0:
        if exponent % 2 == 1:
            result = (result * base) % modulus
        exponent = exponent // 2
        base = (base ** 2) % modulus
    return result

HOST = "socket.cryptohack.org"
PORT = 13373

tn = telnetlib.Telnet(HOST, PORT)

def readline():
    return tn.read_until(b"\n")

def json_recv():
    line = readline().decode()
    line = line[line.find("{"):]
    return json.loads(line)

def json_send(hsh):
    request = json.dumps(hsh).encode()
    tn.write(request)

def getInitMessage():
    d = json_recv()
    p = int(d["p"], 16)
    g = int(d["g"], 16)
    A = int(d["A"], 16)
    return p, g, A

def createSharedSecret(pk, sk, p):
    return modexp(pk, sk, p)

def makePK(sk, g, p):
    return modexp(g, sk, p)

def sendToBob(p, g, A):
    d = {"p": p, "g": g, "A": A}
    json_send(d)

def getFromB():
    d = json_recv()
    B = int(d["B"], 16)
    return B

def sendToAlice(B):
    d = {"B": B}
    json_send(d)

def getIvAndFlag():
    d = json_recv()
    iv = d["iv"]
    encrypted_flag = d["encrypted"]
    return iv, encrypted_flag

# Desde A
p, g, A = getInitMessage()
# Desde B
B = getFromB()
# Desde A
iv, enc_flag = getIvAndFlag()

# A Bob
sendToBob(hex(p), hex(A), "0x01")
# Desde B
sharedK = getFromB()

# Importación del módulo decrypt (asegúrate de tenerlo)
from decrypt import decrypt_flag

# Descifrado de la bandera
flag = decrypt_flag(sharedK, iv, enc_flag)
print(flag)


ModuleNotFoundError: No module named 'decrypt'

In [8]:
import telnetlib
import json

def modexp(base, exponent, modulus):
    result = 1
    base = base % modulus
    while exponent > 0:
        if exponent % 2 == 1:
            result = (result * base) % modulus
        exponent = exponent // 2
        base = (base ** 2) % modulus
    return result

def communicate_with_bob():
    HOST = "socket.cryptohack.org"
    PORT = 13373

    tn = telnetlib.Telnet(HOST, PORT)

    def readline():
        return tn.read_until(b"\n")

    def json_recv():
        line = readline().decode()
        line = line[line.find("{"):]
        return json.loads(line)

    def json_send(hsh):
        request = json.dumps(hsh).encode()
        tn.write(request)

    def get_init_message():
        d = json_recv()
        p = int(d["p"], 16)
        g = int(d["g"], 16)
        A = int(d["A"], 16)
        return p, g, A

    def create_shared_secret(pk, sk, p):
        return modexp(pk, sk, p)

    def make_pk(sk, g, p):
        return modexp(g, sk, p)

    def send_to_bob(p, g, A):
        d = {"p": p, "g": g, "A": A}
        json_send(d)

    def get_from_b():
        d = json_recv()
        B = int(d["B"], 16)
        return B

    def send_to_alice(B):
        d = {"B": B}
        json_send(d)

    def get_iv_and_flag():
        d = json_recv()
        iv = d["iv"]
        encrypted_flag = d["encrypted"]
        return iv, encrypted_flag

    # From A
    p, g, A = get_init_message()
    # To Bob
    send_to_bob(hex(p), hex(A), "0x01")
    # From B
    B = get_from_b()
    # To A
    send_to_alice(B)
    # From A
    iv, enc_flag = get_iv_and_flag()

    # Now you can communicate with Bob or perform any necessary actions

    tn.close()

# Call the function to start communication with Bob
communicate_with_bob()


In [13]:
pip install pycryptodome sympy


Note: you may need to restart the kernel to use updated packages.


In [21]:
from Crypto.Util.number import long_to_bytes
from sympy import GF, Matrix, MatrixGroup
from sympy.combinatorics import Permutation, PermutationGroup

P = 2
N = 50
E = 31337

FLAG = b'crypto{??????????????????????????}'

def bin_to_bytes(s):
    all_bytes = [s[i:i+8] for i in range(0, len(s), 8)]
    return b''.join(long_to_bytes(int(byte, 2)) for byte in all_bytes)

# Placeholder for the missing function
def load_matrix(file_name):
    # Your implementation to load the matrix from the file goes here
    # You need to return a Matrix object
    pass

# https://en.wikipedia.org/wiki/General_linear_group#Over_finite_fields
def find_GL_order():
    order = 1
    k = 0
    for k in range(N):
        order *= (P**N - P**k)
    
    assert order == MatrixGroup(Matrix.eye(N), GF(P)).order()
    
    return order

# Placeholder for the missing variable
# You need to define the missing variable C
C = load_matrix('flag.enc')

GLorder = find_GL_order()

# Placeholder for the missing variable
# You need to define the missing variable D
D = pow(E, -1, GLorder)   # either GLorder or C.multiplicative_order()

M = C**D

# Placeholder for the missing variable
# You need to define the missing variable cols
cols = M.columns()[:(len(FLAG)*8//50)+1]

bin_flag = ''.join([str(bit) for col in cols for bit in col])

print(bin_to_bytes(bin_flag))


ImportError: cannot import name 'MatrixGroup' from 'sympy' (/Users/santiagotene/anaconda3/lib/python3.11/site-packages/sympy/__init__.py)

In [23]:
from Crypto.Cipher import AES
from Crypto.Hash import SHA256
from Crypto.Util.Padding import unpad

P = 2
N = 150
proof.arithmetic(False)

def load_matrix(fname):
    data = open(fname, 'r').read().strip()
    rows = [list(map(int, row)) for row in data.splitlines()]
    return Matrix(GF(P), rows)

G = load_matrix('generator.txt')
A_pub = load_matrix('alice.pub')
B_pub = load_matrix('bob.pub')

f = G.charpoly()

X = []
M = []
for g,e in f.factor():
    assert e == 1
    K = GF(2^g.degree(), x, modulus=g, impl='pari_ffelt')
    a = g.roots(K)[0][0]
    w = (G - a*1).right_kernel_matrix().rows()[0]
    V = [vector([0]*i + [1] + [0]*(N-1-i)) for i in range(150)]
    P = Matrix(K, [w] + V[:-1]).transpose()
    assert P.row_space().dimension() == N
    J_ = ~P * A_pub * P
    X.append(int(J_[0][0].log(a)))
    M.append(K.multiplicative_generator().multiplicative_order())

A_priv = crt(X, M)
shared_secret = B_pub^A_priv

iv = '43f14157442d75142d0d4993e99a9582'
ciphertext = '22abc3b347ffef55ec82488e5b4a338da5af7ef1918ac46f95029a4d94ace4cb2700fa9aeb31e6a4facee2601e99dabd6f9a81494c55f011e9227c9a6ae8d802'
KEY_LENGTH = 128
def derive_aes_key(M):
    mat_str = ''.join(str(x) for row in M for x in row)
    return SHA256.new(data=mat_str.encode()).digest()[:KEY_LENGTH]
key = derive_aes_key(shared_secret)
cipher = AES.new(key, AES.MODE_CBC, bytes.fromhex(iv))
flag = cipher.decrypt(bytes.fromhex(ciphertext))
print(unpad(flag, 16).decode())

NameError: name 'proof' is not defined